<a href="https://colab.research.google.com/github/dreamingv-oid/CS290/blob/main/Project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.dates as mdates
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix




In [12]:
def Attribute_selection_method(df, target_var, criterion):
    # Splitting features and target variable
    X = df.drop(target_var, axis=1)
    y = df[target_var]

    # Identify categorical columns
    categorical_cols = X.select_dtypes(include=['object', 'category']).columns

    # One-hot encode categorical features
    if len(categorical_cols) > 0:
        X = pd.get_dummies(X, columns=categorical_cols)

    # Use DecisionTreeClassifier instead of DecisionTreeRegressor
    if criterion == "entropy":
        model = DecisionTreeClassifier(criterion="entropy")  # Using entropy criterion
    elif criterion == "gini":
        model = DecisionTreeClassifier(criterion="gini")  # Using gini criterion
    else:
        return "Invalid criterion"

    # Fit the model to the data
    model.fit(X, y)

    # Get feature importances from the decision tree model
    importances = model.feature_importances_

    # Find the best feature with the highest importance
    best_feature = X.columns[np.argmax(importances)]

    return best_feature

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Step 1: Data Preprocessing with One-Hot Encoding
def preprocess_data(df):
    # Separate features and target
    X = df.drop('satisfaction', axis=1)
    y = df['satisfaction']

    # Identify categorical columns
    categorical_cols = X.select_dtypes(include='object').columns.tolist()

    # Target label encoding (satisfied or dissatisfied)
    target_encoder = LabelEncoder()
    y = target_encoder.fit_transform(y)  # Binary encoding for the target variable

    return X, y, categorical_cols, target_encoder

# Load your dataset (assuming it's stored as 'airline_dataset.csv')
df = pd.read_csv('https://raw.githubusercontent.com/dreamingv-oid/CS290/main/train.csv') # Use raw.githubusercontent.com to access raw data
df.dropna(inplace = True)

# Preprocess the data
X, y, categorical_cols, target_encoder = preprocess_data(df)

# Step 2: Define the One-Hot Encoder for Categorical Columns
# Create a ColumnTransformer that will One-Hot Encode the categorical columns
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)  # OneHotEncode categorical columns
    ], remainder='passthrough')  # Keep other columns (e.g., numerical columns) as is

# Step 3: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Build a Pipeline with One-Hot Encoding and DecisionTreeClassifier
clf = Pipeline(steps=[
    ('preprocessor', preprocessor),  # Preprocess the data
    ('classifier', DecisionTreeClassifier(random_state=42))  # Train the decision tree classifier
])

# Step 5: Cross-Validation and Model Training
cv_scores = cross_val_score(clf, X_train, y_train, cv=5)  # 5-fold cross-validation
print(f"Cross-Validation Accuracy: {cv_scores.mean():.4f}")

# Train the model on the full training data
clf.fit(X_train, y_train)

# Step 6: Model Evaluation on the Test Set
y_pred = clf.predict(X_test)

# Step 7: Performance Assessment
print("Accuracy Score:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=target_encoder.classes_))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


Attribute_selection_method(df,'satisfaction','gini')


Cross-Validation Accuracy: 0.9430
Accuracy Score: 0.9473912833630966

Classification Report:
                          precision    recall  f1-score   support

neutral or dissatisfied       0.95      0.95      0.95     11655
              satisfied       0.94      0.94      0.94      9064

               accuracy                           0.95     20719
              macro avg       0.95      0.95      0.95     20719
           weighted avg       0.95      0.95      0.95     20719


Confusion Matrix:
 [[11107   548]
 [  542  8522]]


'Online boarding'